In [1]:
import cv2
import numpy as np

In [2]:
def lucas_kanade(img1, img2, p0):
    if p0.shape[-2:] != (1, 2):
        print(f'p0.shape имеет не правильный формат, должно быть (-1, 1, 2), а он {p0.shape}')
        exit()

    lk_params = dict(
        winSize=(50,50),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )

    #img1 = cv2.medianBlur(img1, 5)
    #img2 = cv2.medianBlur(img2, 5)
 
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    p1, st, err = cv2.calcOpticalFlowPyrLK(
        img1, img2, p0, None, **lk_params
    )

    # Пока не буду убирать пиксели которые не двигаются, так как возможно 
    # на других кадрах они подвинутся !!!
    # good_new = p1[st == 1]
    # good_old = p0[st == 1] 

    u = p1 - p0

    return p1, u

In [3]:
def construction_grid(a:int , b:int, n_x:int, n_y:int):
    # a - пикселей по y, b - пикселей по x, 
    # n_x - количество точек по x, n_y - количество точек по y

    points = np.zeros((n_x * n_y, 2), dtype='f')
    
    yw = (a % n_y) // 2
    step_y = a // n_y
    xw = (b % n_x) // 2
    step_x = b // n_x
    print(f'yw = {yw} \nxw = {xw} \nstep_y = {step_y} \nstep_x = {step_x}')

    k = 0

    for j in range(xw, b - xw, step_x):
        for i in range(yw, a - yw, step_y):
            # img[i][j] = black
            points[k] = [np.float32(j), np.float32(i)]
            k += 1
            
    points = points.reshape(points.shape[0], 1, points.shape[1])

    return points

In [8]:
import cv2
import numpy as np

name_video = '_Test1 - alecsandr27000.mp4'

cap = cv2.VideoCapture(name_video)

if not cap.isOpened():
    print("Ошибка открытия видеофайла")
    exit()

ret, frame_old = cap.read()
# ret, frame_new = cap.read()
a, b, r = frame_old.shape
p0 = construction_grid(a, b, 50, 50)
color = np.ones((100001, 3)) * 255

out_video_name = '_out_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width, frame_height = b, a
out_video = cv2.VideoWriter(out_video_name, fourcc, 30.0, (frame_width, frame_height))

mask = np.zeros_like(frame_old)
# lucas_kanade(frame_old, frame_new, p0)
while True:
    ret, frame_new = cap.read()
    
    if not ret:
        break

    p1, ui = lucas_kanade(frame_old, frame_new, p0)

    for i, (new, old) in enumerate(zip(p1, p0)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 1)
            frame_new = cv2.circle(frame_new, (int(a), int(b)), 3, color[i].tolist(), -1)

    img = cv2.add(frame_new, mask)
    out_video.write(img)

    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cap.release()
out_video.release()
cv2.destroyAllWindows()

yw = 15 
xw = 10 
step_y = 21 
step_x = 38


# Result

In [1]:
def lucas_kanade(img1, img2, p0):
    if p0.shape[-2:] != (1, 2):
        print(f'p0.shape имеет не правильный формат, должно быть (-1, 1, 2), а он {p0.shape}')
        exit()

    lk_params = dict(
        winSize=(50,50),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.04),
    )

    img1 = cv2.medianBlur(img1, 5)
    img2 = cv2.medianBlur(img2, 5)
 
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    p1, st, err = cv2.calcOpticalFlowPyrLK(
        img1, img2, p0, None, **lk_params
    )

    # Пока не буду убирать пиксели которые не двигаются, так как возможно 
    # на других кадрах они подвинутся !!!
    # good_new = p1[st == 1]
    # good_old = p0[st == 1] 

    u = p1 - p0

    return p1, u

In [2]:
def construction_grid(a:int , b:int, n_x:int, n_y:int):
    # a - пикселей по y, b - пикселей по x, 
    # n_x - количество точек по x, n_y - количество точек по y

    points = np.zeros((n_x * n_y, 2), dtype='f')
    
    yw = (a % n_y) // 2
    step_y = a // n_y
    xw = (b % n_x) // 2
    step_x = b // n_x
    print(f'yw = {yw} \nxw = {xw} \nstep_y = {step_y} \nstep_x = {step_x}')

    k = 0

    for j in range(xw, b - xw, step_x):
        for i in range(yw, a - yw, step_y):
            # img[i][j] = black
            points[k] = [np.float32(j), np.float32(i)]
            k += 1
            
    points = points.reshape(points.shape[0], 1, points.shape[1])

    return points

In [4]:
def Calc_defor_for_one_cell(O1:list, O2:list, O3:list, U1:list, U2:list, U3:list) -> list:
    
    x1, y1 = O1[0], O1[1]
    x2, y2 = O2[0], O2[1]
    x3, y3 = O3[0], O3[1]
    
    Ux1, Uy1 = U1[0], U1[1]
    Ux2, Uy2 = U2[0], U2[1]
    Ux3, Uy3 = U3[0], U3[1]
    
    F = np.array([[1, x1, y1],
                  [1, x2, y2],
                  [1, x3, y3]])
    A = np.linalg.det(F)
    
    d = np.array([Ux1, Uy1, Ux2, Uy2, Ux3, Uy3]).reshape((6, ))
    
    B = np.array([[y2 - y3, 0, y3 - y1, 0, y1 - y2, 0],
                  [0, x3 - x2, 0, x1 - x3, 0, x2 - x1],
                  [x3 - x2, y2 - y3, x1 - x3, y3 - y1, x2 - x1, y1 - y2]])
    
    E = B @ d / A
    
    return E


def Calc_defor_for_all_cells(Coon_table:list, Coord_table:list, Movement:list)->list:
    E = np.zeros((len(Coon_table), 3))
    
    for O_id in range(len(Coon_table)):
        O1_id, O2_id, O3_id = int(Coon_table[O_id][0]), int(Coon_table[O_id][1]), int(Coon_table[O_id][2])
        
        O1 = Coord_table[O1_id]  
        O2 = Coord_table[O2_id]
        O3 = Coord_table[O3_id]
        U1, U2, U3 = Movement[O1_id], Movement[O2_id], Movement[O3_id]
        
        E[O_id] = Calc_defor_for_one_cell(O1, O2, O3, U1, U2, U3)
        
    return E

In [5]:
def take_E_coord(Coon_table:list, Coord_table:list)->list:
    Coord_of_E = np.zeros((len(Coon_table), 2))
    
    for O_id in range(len(Coon_table)):
        O1_id, O2_id, O3_id = int(Coon_table[O_id][0]), int(Coon_table[O_id][1]), int(Coon_table[O_id][2])
        
        O1, O2, O3 = Coord_table[O1_id], Coord_table[O2_id], Coord_table[O3_id] 
        
        Coord_of_E[O_id] = (np.array(O1) + np.array(O2) + np.array(O3)) / 3 
        
    return Coord_of_E

In [7]:
import cv2
import numpy as np
from _dat_reader import read_connectivity_table, read_coordinates_table

name_video = '_Test1 - alecsandr27000.mp4'

cap = cv2.VideoCapture(name_video)

if not cap.isOpened():
    print("Ошибка открытия видеофайла")
    exit()


for i in range(14):
    ret, frmae_old = cap.read()

ret, frame_old = cap.read()

# ret, frame_new = cap.read()
a, b, r = frame_old.shape
# p0 = construction_grid(a, b, 50, 50)
# p0 = (np.load('nodes_for_ilsur.npy')).astype(np.float32)

p0 = (np.load('_ravil_coord_table.npy')).astype(np.float32)
conn_table = np.load('_ravil_conn_table.npy')

# filename = 'triangle_mesh.dat'
# conn_table = read_connectivity_table(filename)
# node_num, p0 = read_coordinates_table(filename)
p0 = p0.reshape(p0.shape[0], 1, 3)[:, :, :2].astype(np.float32)


# Переворот по x
# p0[:,:,0] *= (-1)
# p0[:,:,0] += b

color = np.ones((100001, 3)) * 255

mask = np.zeros_like(frame_old)
# lucas_kanade(frame_old, frame_new, p0)

test_dict = {}

# print('conn_table = ', conn_table)
# print('p0 = ', p0.shape)

out_video_name = '_out_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width, frame_height = b, a
out_video = cv2.VideoWriter(out_video_name, fourcc, 30.0, (frame_width, frame_height))

e_xx = []
e_xx_coord = []
p1_xx_reshape = []

l = 0
while True:
    l += 1
    ret, frame_new = cap.read()
    
    if not ret:
        break

    p1, ui = lucas_kanade(frame_old, frame_new, p0)

    # MKE
    p1_reshape = np.squeeze(p1)
    p0_reshape = np.squeeze(p0)
    ui_reshape = np.squeeze(ui)
    
    
    e = Calc_defor_for_all_cells(conn_table, p0_reshape, ui_reshape)

    e_coord = take_E_coord(conn_table, p1_reshape)

    e_xx.append(e[:, 0])
    e_xx_coord.append(e_coord)
    p1_xx_reshape.append(p1_reshape)

    print(np.max(e, axis=0))
    print(np.min(e, axis=0))

    #
    #
    # for i, (new, old) in enumerate(zip(p1, p0)):
    #     k += 1
    #     a, b = new.ravel()
    #     c, d = old.ravel()

    #     #
    #     if k == 108 or k == 201 or k == 508 or k == 908 or \
    #         k == 1094 or k == 1203 or k == 1206 or k == 1322 or \
    #             k == 1632 or k == 1936 or k == 2026:
    #         mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), [0, 0, 255], 1)
    #         frame_new = cv2.circle(frame_new, (int(a), int(b)), 3, [0, 0, 255], -1)

    #         if k not in test_dict.keys():
    #             test_dict[k] = [ui[k]]
    #         else:
    #             test_dict[k].append(ui[k])
            
    #         continue
    #     #

    #     mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 1)
    #     frame_new = cv2.circle(frame_new, (int(a), int(b)), 3, color[i].tolist(), -1)

    for point in e_coord:
        frame_new = cv2.circle(frame_new, (int(point[0]), int(point[1])), 4, [0, 0, 255], -1)

    out_video.write(frame_new)
    #img = cv2.add(frame_new, mask)
    #cv2.imshow("frame", img)
    #out_video.write(img)

    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break


print(l)
out_video.release()
cap.release()
cv2.destroyAllWindows()

[0.00748291 0.05686398 0.11059608]
[-0.00789185 -0.1371973  -0.09081688]
[0.00834045 0.04016724 0.14333344]
[-0.01186981 -0.12376194 -0.19211311]
[0.01174774 0.07274303 0.12150383]
[-0.00878906 -0.06084614 -0.14328156]
[0.01755919 0.06886139 0.14559174]
[-0.01581969 -0.08615341 -0.17252579]
[0.01450043 0.05376434 0.12425079]
[-0.01537652 -0.11714554 -0.21078796]
[0.01149139 0.03690453 0.1560585 ]
[-0.02063007 -0.07702942 -0.26265488]
[0.02933769 0.06157379 0.21965256]
[-0.02152214 -0.11901054 -0.41258163]
[0.03832669 0.06572723 0.34538498]
[-0.02795105 -0.10100117 -0.62630844]
[0.05041699 0.07882843 0.4661705 ]
[-0.03511963 -0.09190483 -0.78905563]
[0.05736938 0.12942829 0.70534668]
[-0.04827271 -0.17268009 -1.08186035]
[0.09529953 0.09382019 1.02341156]
[-0.05936279 -0.11172485 -1.3367691 ]
[0.10364456 0.10613251 1.21511154]
[-0.0753479  -0.1460083  -1.68380966]
[0.17607365 0.09580326 1.55542145]
[-0.0940918  -0.13759003 -2.10878906]
[0.18107662 0.13865204 1.97874146]
[-0.12219849 -0.